Import and set up FORMULA executor in Jupyter notebook

In [1]:
import formulallm.formula as f

In [2]:
f.help()

apply (ap)             - Start an apply task. Use: apply transformstep
confhelp (ch)          - Provides help about module configurations and settings
core (cr)              - Prints reduced rule set for domains / transforms. Use: core module_name
del (d)                - Deletes a variable. Use: del var.
det (dt)               - Prints details about the compiled module with the given name. Use: det modname
downgrade (dg)         - Attempts to downgrade a (partial) model to Formula V1. Use: downgrade module_name
exit (x)               - Exits the interface loop.
extract (ex)           - Extract and install a result. Use: extract (app_id | solv_id n) output_name [render_class render_dll]
generate (gn)          - Generate C# data model. Use: generate modname
help (h)               - Prints this message.
interactive (int)      - use: interactive [on | off], will stop interactive prompting (useful for automated tests)
list (ls)              - Lists environment objects. Use: ls [vars | prog

Import `ParserDSL.4ml` source file and `GenericDataParser` domain that provides a generic DSL for parsing data and will be extended to model a concrete parser in FORMULA language.

In [3]:
code = f.load("./data/parser/ParserDSL.4ml")

(Compiled) ParserDSL.4ml
0.27s.


In [4]:
f.details("GenericDataParser")


Reduced form
domain GenericDataParser
{
  Status ::= { INIT, READ, UPDATE, TERMINATE, DONE }.
  Byte ::= new (input: InputData, val: Integer, pos: Integer).
  InputData ::= new (name: String, data: String, length: Integer).
  State ::= new (inputData: InputData, status: Status, curPos: Integer, offset: Integer).
  Transition ::= new (from: State, to: State).
  IntermediateResult ::= new (name: String, context: State, derivedResult: String + Integer + { NULL }).
  NextOffset ::= new (context: State, offset: Integer).
  NextPos ::= new (context: State, pos: Integer).
  OverRead ::= new (context: State).
  CanReadMore ::= new (context: State).
  ContinueCondition ::= new (context: State).

  OverRead(State(inputData, UPDATE, pos, offset))
    :-
      OverRead(State(inputData, READ, pos, offset)).

  OverRead(state)
    :-
      state is State(InputData(_, _, length), _, pos, offset), pos + offset >= length.

  CanReadMore(State(inputData, UPDATE, pos, offset))
    :-
      CanReadMore(St

Import the following 4 files and feed them to the LLM:
1. A buggy C program `untar1.c` from ClamAV
2. A generic DSL for data parsing in `ParserDSL.4ml`
3. FORMULA documentation `formula.pdf`
4. A prompt with specific instructions in `prompt.txt` for the agent to model a Tar parser in FORMULA language.

In [5]:
from formulallm.agents import FormulaLLMAgent
from formulallm.utils.extraction import pdf2text, file2text
text_code_in_c = file2text("./data/parser/untar1.c")
text_parser_dsl = file2text("./data/parser/ParserDSL.4ml")
text_formula_doc = pdf2text("./data/parser/formula.pdf")
text_prompt = file2text("./data/parser/prompt.txt")
print(text_prompt)

1. Understand the C code in untar1.c, FORMULA documentation and ParserDSL.4ml. 
2. Model untar1.c in FORMULA by extending the `GenericDataParser` domain in ParserDSL.4ml. 
3. Do not change the type definition in `GenericDataParser` domain such as Status and IntermediateResult.
4. Use strJoin() in FORMULA to concatenate strings.
5. Model `skipEntry` as IntermediateResult in FORMULA and reflect how its value affects the parsing.
6. Model how `skipEntry` is computed from other IntermediateResults or the current read.


Interactively and iteratively run the LLM agent to model a Tar parser in FORMULA language with additional user input until the next user input is `q` or `quit`.

In [7]:
agent = FormulaLLMAgent(model="gpt-4o")
files = [text_code_in_c, text_parser_dsl, text_formula_doc]
agent.run(text_prompt, files)

Starting conversation with the agent. Type 'exit' to end the session.
Getting back results from initial prompt...
Assistant: To model the `untar1.c` code in FORMULA by extending the `GenericDataParser` domain in `ParserDSL.4ml`, we need to focus on the key aspects of the C code that are relevant to parsing and how they can be represented in FORMULA. Here's a step-by-step approach to achieve this:

1. **Understand the C Code**: The `untar1.c` code is responsible for reading and extracting files from a tar archive. It reads blocks of data, checks for file types, and handles different cases like directories, symbolic links, and special files. The `skipEntry` variable is used to determine whether a particular entry should be skipped based on its type or if there are errors in reading the size.

2. **Identify Key Elements for FORMULA**:
   - **Status**: Represents the state of the parsing process (e.g., INIT, READ, UPDATE, DONE).
   - **IntermediateResult**: Represents intermediate parsing 

User:  exit


Goodbye!


Improve the generated parser DSL in FORMULA and load it into FORMULA executor.

In [8]:
code = f.load("./data/parser/GeneratedTar.4ml")

(Compiled) GeneratedTar.4ml
(Compiled) ParserDSL.4ml
1.02s.


Execute a malicious input model to show that the evaluation of `parsingDone` returns false when the parser gets stuck in an infinite loop and will never reach the end of the input.

In [9]:
f.query("maliciousInput", "parsingDone")

Parsing text took: 1
Visiting text took: 0
Started query task with Id 0.
0.47s.


In [11]:
f.list()


Environment variables

Programs in file root
 +-- /
  +-- Users
   +-- qishenzhang
    +-- projects
     +-- FormulaLLMPY
      +-- examples
       +-- data
        +-- parser [2 file(s)]
        | GeneratedTar.4ml
        | ParserDSL.4ml

Programs in env root
 +-- /

All tasks
 Id | Kind  | Status | Result |     Started      | Duration 
----|-------|--------|--------|------------------|----------
 0  | Query |  Done  | false  | 2/5/2025 4:10 PM |  0.33s   
0.00s.
